## Setup

### Import NuGet packages

In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta2"
#r "nuget: Microsoft.Extensions.Configuration, 7.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Binder, 7.0.0"
#r "nuget: Microsoft.Extensions.Configuration.Json, 7.0.0"
#r "nuget: Microsoft.Extensions.Configuration.UserSecrets, 7.0.0"

#!import config/Env.cs


Installed Packages Microsoft.Extensions.Configuration, 7.0.0 Microsoft.Extensions.Configuration.Binder, 7.0.0 Microsoft.Extensions.Configuration.Json, 7.0.0 Microsoft.Extensions.Configuration.UserSecrets, 7.0.0 Microsoft.SemanticKernel, 1.0.0-beta2

### Add using statements

In [ ]:
using System.IO;
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.Planning;
using Microsoft.SemanticKernel.Plugins.Core;
using Microsoft.SemanticKernel.AI.ChatCompletion;
using Microsoft.SemanticKernel.Planners;

string pluginsDirectory = Path.Combine(Directory.GetCurrentDirectory(), "plugins");

## 1 Basic function running

### 1.1 Run native function

In [16]:
// Create a simple kernel with a single plugin
IKernel kernel = new KernelBuilder()
    //.WithFunctions(new TimePlugin(), "TimePlugin")
    .Build();

// TODO: this should be part of the kernel builder
kernel.ImportFunctions(new TimePlugin(), "TimePlugin");

KernelResult result = await kernel.RunAsync(
    kernel.Functions.GetFunction("TimePlugin", "Today")
);

// TODO: need a way to simplify this, for example:
// KernelResult result = await kernel.RunAsync(
//     "TimePlugin.Today"
// );

Console.WriteLine(result.GetValue<string>());

// TODO: need a way to simplify this, for example:
Console.WriteLine(result);

Tuesday, October 17, 2023


### 1.2 Running functions from OpenAI plugins

### 1.3 Running a semantic function

In [4]:
string AzureOpenAIDeploymentName = Env.Var("AzureOpenAI:ChatCompletionDeploymentName")!;
string AzureOpenAIEndpoint = Env.Var("AzureOpenAI:Endpoint")!;
string AzureOpenAIApiKey = Env.Var("AzureOpenAI:ApiKey")!;

// Create a kernel with an AI service
IKernel kernel = new KernelBuilder()
    .WithAzureChatCompletionService(
        AzureOpenAIDeploymentName,  // The name of your deployment (e.g., "gpt-35-turbo")
        AzureOpenAIEndpoint,        // The endpoint of your Azure OpenAI service
        AzureOpenAIApiKey           // The API key of your Azure OpenAI service
    )
    //.WithSemanticFunctionsFromDirectory(pluginsDirectory, "WriterPlugin")
    .Build();

// TODO: This should be part of the kernel builder
kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "WriterPlugin");

KernelResult result = await kernel.RunAsync(
    "Hello world",
    kernel.Functions.GetFunction("WriterPlugin", "ShortPoem")
);

// TODO: need a way to simplify this
// KernelResult result = await kernel.RunAsync(
//     "WriterPlugin.ShortPoem"
//     context: { "input": "Hello world" }
// );

Console.WriteLine(result.GetValue<string>());

// TODO: need a way to simplify this, for example:
Console.WriteLine(result);

Hello world, how do you do?
I'm just a program, not a kangaroo.
I'm here to help, to answer your call,
So don't be shy, just give me a holler! 

But if you're feeling down and blue,
And need a laugh or two,
Just remember this silly rhyme,
And know that everything will be just fine!


## 1.4 Running the _same_ semantic function with different runtime settings

In [14]:
string AzureOpenAIDeploymentName = Env.Var("AzureOpenAI:ChatCompletionDeploymentName")!;
string AzureOpenAIEndpoint = Env.Var("AzureOpenAI:Endpoint")!;
string AzureOpenAIApiKey = Env.Var("AzureOpenAI:ApiKey")!;

// TODO: Allow creation of semantic function w/o a kernel
// SemanticFunction function = new SemanticFunction();

IKernel kernel_1 = new KernelBuilder()
    .WithAzureChatCompletionService(
        AzureOpenAIDeploymentName,  // The name of your deployment (e.g., "gpt-35-turbo")
        AzureOpenAIEndpoint,        // The endpoint of your Azure OpenAI service
        AzureOpenAIApiKey           // The API key of your Azure OpenAI service
    )
    // TODO: Should be able to import existing function
    //.WithFunctions(function)
    .Build();

IKernel kernel_2 = new KernelBuilder()
    .WithAzureChatCompletionService(
        "gpt-4",
        AzureOpenAIEndpoint,        // The endpoint of your Azure OpenAI service
        AzureOpenAIApiKey           // The API key of your Azure OpenAI service
    )
    // TODO: Should be able to import existing function with model overrides
    //.WithFunctions(function, new ModelOverrides { ModelId = "gpt-4", Temperature = 0.5 })
    .Build();

// TODO: This should be part of the kernel builder
kernel_1.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "WriterPlugin");
kernel_2.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "WriterPlugin");

KernelResult result_1 = await kernel_1.RunAsync(
    "Hello world",
    kernel_1.Functions.GetFunction("WriterPlugin", "ShortPoem")
);

KernelResult result_2 = await kernel_2.RunAsync(
    "Hello world",
    kernel_2.Functions.GetFunction("WriterPlugin", "ShortPoem")
);

// TODO: should also have a way override during RunAsync
// KernelResult result_3 = await kernel.RunAsync(
//     "WriterPlugin.ShortPoem input"
//     context: { "input": "Hello world" }
//     modelOverrides: new ModelOverrides { Function:"WriterPlugin.ShortPoem", ModelId = "gpt-4", Temperature = 0.5 }
// );

// Compre the results
Console.WriteLine("Result 1:\n" + result_1.GetValue<string>());
Console.WriteLine("\nResult 2:\n" + result_2.GetValue<string>());

Result 1:
Hello world, how do you do?
I'm just a program, it's nice to meet you!
I may not have arms or legs or a face,
But I can help you with tasks at a rapid pace.

I'll greet you each time you start up your machine,
And help you navigate through the digital scene.
So don't be shy, give me a shout,
Hello world, let's figure things out!

Result 2:
There once was a phrase so bold,
That in coding, it often was told,
"Hello world," it would say,
In a cheeky display,
As the screen lit up bright and consoled.


### 1.5 Running image generation 

### 1.6 Running audio generation

### 1.7 Streaming functions

## 2 Manually chaining functions together

### 2.1 Using the kernel

In [15]:
string AzureOpenAIDeploymentName = Env.Var("AzureOpenAI:ChatCompletionDeploymentName")!;
string AzureOpenAIEndpoint = Env.Var("AzureOpenAI:Endpoint")!;
string AzureOpenAIApiKey = Env.Var("AzureOpenAI:ApiKey")!;

IKernel kernel = new KernelBuilder()
    .WithAzureChatCompletionService(
        AzureOpenAIDeploymentName,  // The name of your deployment (e.g., "gpt-35-turbo")
        AzureOpenAIEndpoint,        // The endpoint of your Azure OpenAI service
        AzureOpenAIApiKey           // The API key of your Azure OpenAI service
    )
    //.WithSemanticFunctionsFromDirectory(pluginsDirectory, "WriterPlugin")
    .Build();

// TODO: This should be part of the kernel builder
kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "WriterPlugin");
kernel.ImportFunctions(new TimePlugin(), "TimePlugin");

KernelResult result = await kernel.RunAsync(
    kernel.Functions.GetFunction("TimePlugin", "Today"),
    kernel.Functions.GetFunction("WriterPlugin", "ShortPoem")
);

// TODO: need a way to explicitely pass variables, for example:
// KernelResult result = await kernel.RunAsync(
//     "TimePlugin.Today as | today |"
//     "WriterPlugin.ShortPoem today"
// );

Console.WriteLine(result.GetValue<string>());

On Tuesday, October 17, 2023,
The world was filled with glee,
For it was National Dance Like a Chicken Day,
And everyone was ready to sway.

People flapped their wings and did the shuffle,
Some even did it with a truffle,
The streets were filled with clucks and squawks,
And everyone was having a ball, no mocks.

So if you see someone doing the chicken dance,
Don't be shy, just take a chance,
Join in the fun and flap your wings,
And dance like a chicken, it's the best thing!


### 2.2 Using a plan

In [7]:
string AzureOpenAIDeploymentName = Env.Var("AzureOpenAI:ChatCompletionDeploymentName")!;
string AzureOpenAIEndpoint = Env.Var("AzureOpenAI:Endpoint")!;
string AzureOpenAIApiKey = Env.Var("AzureOpenAI:ApiKey")!;

IKernel kernel = new KernelBuilder()
    .WithAzureChatCompletionService(
        AzureOpenAIDeploymentName,  // The name of your deployment (e.g., "gpt-35-turbo")
        AzureOpenAIEndpoint,        // The endpoint of your Azure OpenAI service
        AzureOpenAIApiKey           // The API key of your Azure OpenAI service
    )
    //.WithSemanticFunctionsFromDirectory(pluginsDirectory, "WriterPlugin")
    .Build();

// TODO: This should be part of the kernel builder
kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "WriterPlugin");
kernel.ImportFunctions(new TimePlugin(), "TimePlugin");

// TODO: Create empty constructor for Plan
Plan plan = new Plan("");
plan.AddSteps(
    kernel.Functions.GetFunction("TimePlugin", "Today"),
    kernel.Functions.GetFunction("WriterPlugin", "ShortPoem")
);

// TODO: need a way to pass variables, for example:
// Plan plan = new Plan(
//     "TimePlugin.Today as | today |"
//     "WriterPlugin.ShortPoem today"
// );

KernelResult result = await kernel.RunAsync(plan);


Console.WriteLine(result.GetValue<string>());

On Tuesday, October 17, 2023,
The world was filled with glee,
For it was National Dance Like a Chicken Day,
And everyone was ready to sway.

People flapped their wings and did the shuffle,
Some even did it with a truffle,
The streets were filled with clucks and squawks,
And everyone was having a ball, no mocks.

So if you see someone doing the chicken dance,
Don't be shy, just take a chance,
Join in the fun and flap your wings,
And dance like a chicken, it's the best thing!


### 2.3 Streaming functions with chaining 

### 2.4 Chaining functions with objects

### 2.5 Multi-model chaining

## 3 Using planners

### 3.1 Using sequential planner

In [8]:
string AzureOpenAIDeploymentName = Env.Var("AzureOpenAI:ChatCompletionDeploymentName")!;
string AzureOpenAIEndpoint = Env.Var("AzureOpenAI:Endpoint")!;
string AzureOpenAIApiKey = Env.Var("AzureOpenAI:ApiKey")!;

IKernel kernel = new KernelBuilder()
    .WithAzureChatCompletionService(
        AzureOpenAIDeploymentName,  // The name of your deployment (e.g., "gpt-35-turbo")
        AzureOpenAIEndpoint,        // The endpoint of your Azure OpenAI service
        AzureOpenAIApiKey           // The API key of your Azure OpenAI service
    )
    //.WithSemanticFunctionsFromDirectory(pluginsDirectory, "WriterPlugin")
    //.WithFunctions(new TimePlugin(), "TimePlugin")
    .Build();

// TODO: These should be part of the kernel builder
kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "WriterPlugin");
kernel.ImportFunctions(new TimePlugin(), "TimePlugin");

SequentialPlanner planner = new SequentialPlanner(kernel);
var ask = "Can you write a poem about today's date?";
var plan = await planner.CreatePlanAsync(ask);

KernelResult result = await kernel.RunAsync(plan);

Console.WriteLine(result.GetValue<string>());

Today is Tuesday, what a date!
October 17, 2023, don't be late!
The sun is shining, the birds are singing,
It's a lovely fate, no need for clinging.

The world is spinning, time is flying,
But don't you worry, no need for crying.
Just enjoy the day, have some fun,
And before you know it, it's already done.

So let's raise a glass, and give a cheer,
For this lovely day, that's finally here.
Let's make the most of it, and have a blast,
Because Tuesday, October 17, 2023, is going to be a blast!


### 3.2 Using stepwise planner

In [10]:
string AzureOpenAIDeploymentName = Env.Var("AzureOpenAI:ChatCompletionDeploymentName")!;
string AzureOpenAIEndpoint = Env.Var("AzureOpenAI:Endpoint")!;
string AzureOpenAIApiKey = Env.Var("AzureOpenAI:ApiKey")!;

IKernel kernel = new KernelBuilder()
    .WithAzureChatCompletionService(
        AzureOpenAIDeploymentName,  // The name of your deployment (e.g., "gpt-35-turbo")
        AzureOpenAIEndpoint,        // The endpoint of your Azure OpenAI service
        AzureOpenAIApiKey           // The API key of your Azure OpenAI service
    )
    //.WithSemanticFunctionsFromDirectory(pluginsDirectory, "WriterPlugin")
    //.WithFunctions(new TimePlugin(), "TimePlugin")
    .Build();

// TODO: These should be part of the kernel builder
kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "WriterPlugin");
kernel.ImportFunctions(new TimePlugin(), "TimePlugin");

StepwisePlanner planner = new StepwisePlanner(kernel);
var ask = "Can you write a poem about today's date?";
var plan = planner.CreatePlan(ask);

KernelResult result = await kernel.RunAsync(plan);

Console.WriteLine(result.GetValue<string>());

Here's a short poem about today's date:

It's Tuesday, October 17, 2023,
A day that's just as good as can be,
The sun is shining, the birds are singing,
And the world is just a happy thing.

But wait, what's that I hear you say?
You're feeling down and out today?
Well, don't you worry, don't you fret,
Just take a deep breath and don't forget,

That life is short and time is fleeting,
So let's enjoy this day we're meeting,
Let's laugh and love and have some fun,
And make this Tuesday number one!
